## Análisis de fuente


# Archivos

**monedas.csv**  
Tiene únicamente las monedas que utiliza la plataforma, ARS y USD

**ubicaciones.csv**  
Tabla de ubicaciones genéricas, contiene las columnas  

| XIntelID | Pais      | Estado                  | Localidad | Barrio   | Comuna  |
|----------|-----------|-------------------------|-----------|----------|---------|
| 13242    | Argentina | Bs.As. G.B.A Zona Norte | Tigre     | Nordelta | La Isla |

**tipologías.csv**  
Tipos de propiedades que hay en la plataforma, en cuanto a columnas solo interesa el *nombre* de la tipología. Por ejemplo, "Casas", "Departamentos", "Cocheras", etc.

**medidas.csv**  
Parecería ser la caracterización de un ambiente dentro de una propiedad, tiene un ID de tipología y un nombre, por ejemplo "Living", "Comedor", "Baños", etc.

**inombiliarias.csv**  
Tabla con información de la inmobiliaria que publica la propiedad, no nos interesará mucho para nuestro análisis.

**caracteristicas.csv**  
Listado de características que puede tener un inmueble. Vemos cosas como "Acceso a Internet", "Aire acondicionado", "Gimnasio", "Pileta", etc. Se podría considerar algunas de las características como las **ammenities** que tiene la propiedad. Podríamos a futuro separar cuales lo son y cuales no a mano, ya que son pocas

**características_inmuebles.csv**  
Tabla utilizada para la relacion muchos a muchos que tienen las características y los inmuebles.

**operacion.csv**  
Nos indica el tipo de operación de la propiedad, por ejemplo, si es para Alquiler, Venta, Alquiler Temporal, etc.

**inmuebles.xlsx**  
Archivo con todos los inmuebles publicados actualmente


In [427]:
#python3 -m pip install <library>

# MongoDB Atblas
# username: uade-pfi
# password: vHshDD0jA73LlHAW

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

#Importamos los distintos archivos que vamos a utilizar para generar el DataFrame final
df_inmuebles = pd.read_excel("Source/cabaprop_julio_2023/inmuebles.xlsx", converters={
    "id_ubicacion": int,
    "id_moneda": int,
    "id_operacion": int,
    "id_tipologia": int,
    "inmobiliaria_id": int,
    "precio_dolar": int,
    "precio_peso": int,
    "ambientes": int,
    "dormitorios": int,
    "antiguedad": int
})


In [428]:
import base64

def decode_b64_string(row):
    # Caso de campo vacío, va a ser None
    if pd.isnull(row):
        #print(row)
        return row
    sample = str(row)
    sample = base64.b64decode(sample)
    sample = sample.decode("utf-8")
    
    return sample

In [429]:

# Tenemos algunas filas con info en base64, hacemos un decode para tener el texto
df_inmuebles.descripcion = df_inmuebles.descripcion.apply(decode_b64_string)
df_inmuebles.title = df_inmuebles.title.apply(decode_b64_string)
df_inmuebles.direccion = df_inmuebles.direccion.apply(decode_b64_string)

In [430]:
# Removemos columnas que no nos van a importar para el análisis
# Precio es precio_dolar con algunos ceros de mas, utilizamos precio_dolar y precio_peso directamente

df_inmuebles.drop(["created", "updated", "inmobiliaria_id", "precio", "dpto", "piso_1", "unidadesPiso"], axis=1, inplace=True)
df_inmuebles.rename(columns={"id": "id_inmueble"}, inplace=True)


In [445]:
# Removemos filas que no nos van a servir

# Inmuebles sin ubicación
df_inmuebles = df_inmuebles.loc[pd.notnull(df_inmuebles["id_ubicacion"])]

# Inmuebles sin precio en pesos o dolares
df_inmuebles = df_inmuebles.loc[pd.notnull(df_inmuebles["precio_dolar"]) | pd.notnull(df_inmuebles["precio_peso"])]

# Inmuebles con precio 0 en pesos y dolares
df_inmuebles = df_inmuebles.loc[(df_inmuebles["precio_dolar"]!=0) | (df_inmuebles["precio_peso"]!=0)]

# Inmuebles con precios negativos
df_inmuebles = df_inmuebles.loc[(df_inmuebles["precio_dolar"]>=0) & (df_inmuebles["precio_peso"]>=0)]

Vamos a ahora combinar esta tabla principal con las otras, para poder lograr tener un único dataframe con
toda la información que necesitaremos para el modelo.

In [432]:
# Combinamos ubicación

df_ubicacion = pd.read_csv("Source/cabaprop_julio_2023/ubicaciones.csv")

# Removemos algunas columnas que no nos importan
df_ubicacion.drop(["xintel_id", "updated", "created"], axis=1, inplace=True)

In [433]:
# Realizamos el Merge
df_inmuebles_ubi = pd.merge(df_inmuebles, df_ubicacion, how="left", left_on="id_ubicacion", right_on="id")

# Removemos las columnas que no nos importan o las que son generadas por el merge
df_inmuebles_ubi.drop(["id_ubicacion", "id"], axis=1, inplace=True)

In [434]:
# Combinamos monedas, como ya tenemos en el set de datos precio Peso y Precio dolar, ya tenemmos la información ahí, por
# lo que salteamos el merge

df_inmuebles_ubi.drop(["id_moneda"], axis=1, inplace=True)

In [435]:
# Combinamos operaciones, para ver el tipo de transacción

df_operacion = pd.read_csv("Source/operacion.csv")
df_operacion.drop(["updated", "created"], axis=1, inplace=True)
# Renombramos la columna para que se entienda mejor luego del merge
df_operacion.rename(columns={"name": "tipo_operacion"}, inplace=True)


In [436]:
# Realizamos el Merge
df_inmuebles_ubi_op = pd.merge(df_inmuebles_ubi, df_operacion, how="left", left_on="id_operacion", right_on="id")

# Removemos las columnas que no nos importan o las que son generadas por el merge
df_inmuebles_ubi_op.drop(["id", "id_operacion"], axis=1, inplace=True)

In [437]:
# Combinamos tipologias

df_tipologias = pd.read_csv("Source/cabaprop_julio_2023/tipologias.csv")
df_tipologias.drop(["created", "updated"], axis=1, inplace=True)
df_tipologias.rename(columns={"name": "tipo_propiedad", "tags": "amenities"}, inplace=True)


In [438]:
# Realizamos el Merge
df_inmuebles_ubi_op_tip = pd.merge(df_inmuebles_ubi_op, df_tipologias, how="left", left_on="id_tipologia", right_on="id")
df_inmuebles_ubi_op_tip
# Removemos las columnas que no nos importan o las que son generadas por el merge
df_inmuebles_ubi_op_tip.drop(["id", "id_tipologia"], axis=1, inplace=True)

In [439]:
# Nos queda características con características inmuebles. Esta info ya debería estar en amenities pero podriamos hacer un
# doble check

In [440]:
# Vamos a revisar la info que nos quedo para ver que encontramos

df_final = df_inmuebles_ubi_op_tip.copy(deep=True)

# Encontramos que tipo_propiedad es casi siempre lo mismo que subTipoPropiedad.
# Vamos a quedarnos solo con una de esas columnas
df_final.loc[df_final["subTipoPropiedad"] == "PH", ["subTipoPropiedad"]] = "PH"
df_final.loc[df_final["subTipoPropiedad"] == "DEPARTAMENTO", ["subTipoPropiedad"]] = "Departamentos"

df_final["tipo_propiedad"].fillna(df_final["subTipoPropiedad"], inplace=True)

In [441]:
# Chequeamos que no nos queden vacios en tipo_propiedad que esten llenos en subTipoPropiedad y removemos la columna subTipo
print(df_final[pd.isnull(df_final["tipo_propiedad"]) & pd.notnull(df_final["subTipoPropiedad"])].shape)
df_final.drop(["subTipoPropiedad"], axis=1, inplace=True)

(0, 28)


In [442]:
# El tipo de propiedad es muy importante para nuestro análisis
# Vamos a investigar la descripción de la propiedad para intentar inferir el tipo de la misma

str_dptos = ["Departamento", "Dpto", "Monoambiente", "DEPARTAMENTO", "DPTO"]
str_oficinas = ["Oficina", "OFICINA"]

def analizar_tipo_propiedad(row):
    if any([x in str(row) for x in str_dptos]):
        return "Departamentos"
    elif any([x in str(row) for x in str_oficinas]):
        return "Oficinas"
    elif pd.isnull(row):
        return row
    else:
        return np.nan

# Vamos a investigar tanto el titulo como la descripción de la propiedad para poder llenar mas info
df_final.loc[pd.isnull(df_final["tipo_propiedad"]), ["tipo_propiedad"]] = df_final["title"].apply(analizar_tipo_propiedad)
df_final.loc[pd.isnull(df_final["tipo_propiedad"]), ["tipo_propiedad"]] = df_final["descripcion"].apply(analizar_tipo_propiedad)

# Luego de esto nos quedan solo 15, muchos de esos no teniendo ni titulo, ni descripción, ni info. Los removemos
print(df_final[pd.isnull(df_final["tipo_propiedad"])].shape) 
df_final = df_final[pd.notnull(df_final["tipo_propiedad"])]

(4, 27)


In [443]:

# Filtramos por argentina, ya que es nuestro scope
df_final = df_final[df_final["pais"] == "Argentina"]

# Filtramos por capital federal, ya que es nuestro scope
df_final = df_final[df_final["estado"] == "Capital Federal"]

# Nos quedamos únicamente con los departamentos, PH y casas ya que es nuestro foco inicial
df_final = df_final[df_final["tipo_propiedad"].isin(["Departamentos", "PH", "Casas"])]

# Removemos las filas que no tienen ningun valor en las superficies
df_final.drop(df_final[pd.isnull(df_final["superficie_total"]) & pd.isnull(df_final["superficie_cubierta"]) & pd.isnull(df_final["superficie_descubierta"])].index, inplace=True)

df_final.tipo_operacion.unique()
print(df_final.shape)
print(df_final.tipo_operacion.unique())

df_final.loc[(df_final.tipo_operacion == "Alquiler") | (df_final.tipo_operacion == "Alquiler Temporal")].to_csv("source.csv")

(24245, 27)
['Venta' 'Alquiler' 'Alquiler Temporal' nan 'Permuta']


In [444]:
#Agregar contador de caracteres en la descripción, para ser luego utilizado en la matriz de correlación